# Exploration 06_작사가 인공지능 만들기       


## 이 노드의 루브릭   

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?     
    -> "텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?       
    

2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?     
    -> "특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?"          
    

3. 텍스트 생성모델이 안정적으로 학습되었는가?     
    -> "텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?"   

* 위의 루브릭에 주의하며 작사가 인공지능을 만들어 보자!

### 1. 데이터 가져오기          

- 데이터가 잘 준비되었음을 알 수 있다.

![데이터 준비](./PostingPic/데이터준비.png)

### 2. 데이터 읽어오기    

* 데이터를 raw_corpus에 담는다.(가사를 정제해서 텐서화 시킬 raw_corpus)

In [5]:
import glob
import os

txt_path = os.getenv('HOME')+'/SUBMIT_MISSION_GIT/ex6_Composer/lyrics/*'
txt_list = glob.glob(txt_path)

print("위치 : " ,txt_path)

raw_corpus =[]

for txt_file in txt_list:
    
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        
        #리스트.extend(내용), 리스트 끝에 raw의 모든 항목을 넣는다.
        raw_corpus.extend(raw)
        
print("데이터 크기 : ", len(raw_corpus))
print("Examples : ", raw_corpus[:3])

위치 :  /home/ssac23/SUBMIT_MISSION_GIT/ex6_Composer/lyrics/*
데이터 크기 :  187088
Examples :  ['', '', '[Spoken Intro:]']


* 번외(궁금해서 해봤다)  

- 새로운 파이썬 문법.. raw_corpus.extend(raw)가 나와서 검색해봤더니, append와 비슷한 거라고 한다.   
- 근데, 어떤 부분이 다른거지? 그걸 알기 위해 한 번 해봤더니...!

![궁금해서 해봤더니](./PostingPic/append할때.png)   


- 위에서 raw는 
  1. lyrics 폴더의 모든 텍스트 리스트(txt_list)에서 가져온     
  2. 하나의 텍스트 파일(txt_file)에서 읽어온 splitlines() 이다.    
---
- 얘를 append 하면 하나의 파일만 읽어오고,    
- 얘를 extend 하면 그 폴더의 모든 파일의 데이터가 읽어와진다.    
---
__WHY?__
> - append : object(그 해당 오브젝트)를 맨 뒤에 추가한다.    
> - extend : iterable객체(리스트, 튜플, 딕셔너리)의 원소를 list에 appending 한다.

### 3. 데이터 정제하기    

* preprocess_sentence() 함수에서 나타난 다양한 스킬들을 통해 데이터를 정제한다.    
* 단, 문장을 토큰화했을 때 토큰의 개수가 15개가 넘어가는 문장은 학습데이터에서 제외한다.

In [9]:
import re
import numpy as np
import tensorflow as tf

### 3-1. 데이터 정제하기

####  혹시 모를 특수문자, 대소문자, 문장부호를 정제하고, 소스 첫단과 끝단에 start와 end를 붙여준다.

In [13]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    
    sentence = re.sub("([?.!,?,¿])", r" \1 ", sentence)
    sentence = re.sub('[" "]+', " ", sentence)
    sentence = re.sub("[^a-zA-Z?.!,¿]+", " ", sentence)
    
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

In [15]:
#정제된 표현을 담을 new_corpus 배열을 선언한다.

new_corpus = []

for sentence in raw_corpus :
    new_corpus.append(preprocess_sentence(sentence))
    
print("정제 후 new_corpus의 길이 : ", len(new_corpus))
new_corpus[3:10]

정제 후 new_corpus의 길이 :  187088


['<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>']

#### 정제한 문장을 토큰화하고, 단어 사전을 만들며, 데이터를 숫자로 변환하기 == "토큰화하기"

In [20]:
def tokenize(corpus):
    
    #순서대로 전체 단어의 개수, 추가할 전처리 로직, out-of vocabulary(사전에 없는 단어를 무엇으로 대체할지)
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words =7000, filters='' , oov_token="<unk>")
    
    #우리가 정제한 corpus를 토큰화하여 사전으로 변환
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    #입력 데이터의 시퀀스 길이를 맞추기 위해 padding 메소드 적용
    #maxlen의 default 값은 None으로, corpus의 가장 긴 문장을 기준으로 시퀀스 길이를 맞춤
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(new_corpus)

[[   2    3    0 ...    0    0    0]
 [   2    3    0 ...    0    0    0]
 [   2 2706 2589 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [   2   23   89 ...    0    0    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fb5f8b62590>


> 첫단은 2(start)로, 뒤에는 '패딩' 이 들어간 것을 확인할 수 있다.

In [29]:
print(len(tensor))
print(tensor)

187088
[[   2    3    0 ...    0    0    0]
 [   2    3    0 ...    0    0    0]
 [   2 2706 2589 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [   2   23   89 ...    0    0    0]
 [   2    7   34 ...    0    0    0]]


#### 토큰화했을때, 너무 긴 문장은 작사하는데 맞지 않을 수 있다. 길이가 15가 넘어가는 데이터는 지우자!        

- 위에서 변환된 텐서를 출력해보았을 때, 187088개의 데이터가 무사히 텐서로 변환되었지만     
- 변환된 텐서의 끝에 공백이 0 이 들어가는 데이터가 너무 많아 문제인 것을 알 수 있다.     
- 모델의 성능에 영향을 미칠 수 있으므로 공백기준 15개 미만의 단어만 tensor 안에 넣어주고,     
- 텐서의 개수를 출력해보도록 한다.

In [39]:
temp_corpus = []

#혹시 몰라서 <start>, <end>를 빼서 계산했다. (15-2(스타트, 엔드))
for sentences in new_corpus:
    if len(sentences.split(' ')) < 13:
        temp_corpus.append(sentences)
        
tensor, tokenizer = tokenize(temp_corpus)
print(len(tensor))

[[   2    3    0 ...    0    0    0]
 [   2    3    0 ...    0    0    0]
 [   2 2569 2160 ...    0    0    0]
 ...
 [   2  677   30 ...  428    3    0]
 [   2    4  124 ...   10    7    3]
 [   2    7   37 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fb68042db90>
142296
